In [1]:
from src import *
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

### data preparation

In [8]:
DF = pd.read_csv('./data/task4.csv', index_col='timestamp', parse_dates=True)
DF = DF.apply(pd.to_numeric)

In [9]:
market = DF.iloc[1::2].reset_index(drop=True) - DF.iloc[::2].reset_index(drop=True)
market.index = DF.index[1::2].date
price = market['price'].cumsum()
df = DF.iloc[1::2]
df = df.drop(columns=['price'])
df.index = df.index.date
df['price'] = price
df

,0_long,1_long,2_short,3_short,4_long,5_long,6_long,7_long,8_long,9_short,...,258_long,259_long,260_long,261_long,262_short,263_short,264_short,265_short,naive_ave_pnl,price
2006-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
2006-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-4.00
2006-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.00
2006-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
2006-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-07,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1207.0,1799.0,8677.0,...,518.0,1970.0,3170.0,420.0,6349.0,4267.0,4169.0,8467.0,3266.097744,-495.50
2025-07-08,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1207.0,1817.0,8677.0,...,518.0,1988.0,3188.0,420.0,6349.0,4267.0,4169.0,8467.0,3267.518797,-491.00
2025-07-09,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1277.0,1887.0,8677.0,...,518.0,1988.0,3258.0,490.0,6349.0,4267.0,4169.0,8467.0,3270.413534,-473.50
2025-07-10,4993.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1277.0,1822.0,8677.0,...,518.0,1988.0,3193.0,425.0,6349.0,4267.0,4169.0,8467.0,3265.281955,-489.75


In [11]:
long_columns = [c for c in df.columns if 'long' in c] + ['naive_ave_pnl', 'price']
short_columns = [c for c in df.columns if 'short' in c] + ['naive_ave_pnl', 'price']
long = df[long_columns]
short = df[short_columns]

### Weights Playing

In [15]:
def get_weights(rt = 0.25, drawdown = 0.25, length = 0.25, volatility = 0.25):
    # rt: drawup_beta, pnl_t, excess_mean_ret, excess_ret, success_rate. --- 5
    # drawdown: mdd_p, ldd_p, cvar_5, drawdown_beta, cdd_percent, mdd_days, ldd_days, calmar --- 8
    # length: days_since_first_trade --- 1
    # volatility: sharpe --- 1

    # order: 'mdd_percent’,’ ldd_percent’, ‘sharpe’:, ‘cdd_percent’, ‘mdd_days'
    # 'ldd_days’, ‘pnl_t’, ‘days_since_first_trade’, ‘drawdown_beta’, ‘drawup_beta’
    # ‘excess_mean_ret’, ‘excess_ret’, cvar_5’, ‘success_rate’, ‘calmar'

    return np.array([drawdown/8, drawdown/8, volatility, drawdown/8, drawdown/8, drawdown/8, rt/5, 
            length, drawdown/8, rt/5, rt/5, rt/5, drawdown/8, rt/5, drawdown/8])


In [18]:
long_metrics = calc_metrics(long)

/Users/vivianguo/Sauma Capital/sauma-group/src/metrics.py:64: RuntimeWarning: invalid value encountered in divide
  sharpe = np.where(rstd == 0, 0, rmean / rstd * np.sqrt(252))
/Users/vivianguo/Sauma Capital/sauma-group/src/metrics.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if dd[i] == 0:
/Users/vivianguo/Sauma Capital/sauma-group/src/metrics.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  while j >= 0 and dd[j] > 0:


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
short_metrics = calc_metrics(short)